In [ ]:
pip install networkx

In [ ]:
pip install haversine

In [ ]:
from script import df_mobpro_brut, arr_marseille, arr_paris, arr_lyon, contours_comm, transport_dict, contours_comm, flux, plot_flux_gradient, plot_flux_gradient_zoom, coord_villes
from script import df_dossier_complet_brut, mairies, df_epci_contours, df_flux_jaune_destination_m,df_flux_rouge_destination_m,df_flux_vert_destination_m, df_flux_rouge_depart_m,df_flux_jaune_depart_m,df_flux_vert_depart_m
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns
import ast
import numpy as np
import matplotlib.colors as colors
import networkx as nx
from scipy.sparse import coo_matrix
from shapely.geometry import Point
from shapely.ops import unary_union
from shapely import wkt
from haversine import haversine

In [ ]:
mairies[['lat', 'lon']] = mairies['Coordonnées'].str.split(',', expand=True).astype(float)
mairies_unique = mairies.drop_duplicates(subset='Code INSEE')

# Calculer le trajet interne
def approx_internal_distance(geom):
    bounds = geom.bounds  # (minx, miny, maxx, maxy)
    point_min = (bounds[1], bounds[0])  # (lat_min, lon_min)
    point_max = (bounds[3], bounds[2])  # (lat_max, lon_max)
    return haversine(point_min, point_max)

contours_comm['distance_intra'] = contours_comm['geometry'].apply(approx_internal_distance)/2 # On divise par 2 la diag pour les intra
distance_intra_dict = contours_comm.set_index('INSEE_COM')['distance_intra'].to_dict() # Dictionnaire des distances intra

# Calculer les trajets entre villes
mairies_unique = mairies.drop_duplicates(subset='Code INSEE') # Garder une seule mairie pour chaque ville
coords_mairies = mairies_unique.set_index('Code INSEE')[['lat', 'lon']].to_dict('index') 

# Fonction pour calculer la distance
def calculer_distance(code_depart, code_arrivee):
    if code_depart == code_arrivee:
        # Trajet intra-communal
        return distance_intra_dict.get(code_depart, 3.0)  # Fallback 3km si pas trouvé
    else:
        # Trajet intercommunal
        coords_depart = coords_mairies.get(code_depart)
        coords_arrivee = coords_mairies.get(code_arrivee)
        
        if coords_depart is None or coords_arrivee is None:
            return None  # Impossible de calculer (missing coords)
        
        return haversine((coords_depart['lat'], coords_depart['lon']),
                         (coords_arrivee['lat'], coords_arrivee['lon']))

df_mobpro_brut['distance_km'] = df_mobpro_brut.apply(
    lambda row: calculer_distance(row['COMMUNE'], row['DCLT']),
    axis=1)


In [ ]:
df_mobpro_brut.head(20)

In [ ]:
def est_mono_departement(liste_departements):
    '''
    On vérifie si les EPCI regroupent forcément des villes qui appartiennent au même département (réponse = non pas systématiquement)
    '''
    liste_departements = ast.literal_eval(liste_departements)
    return len(set(liste_departements)) == 1
    
    
# Appliquer la fonction à chaque ligne et créer une nouvelle colonne
df_epci_contours['MONO_DEP'] = df_epci_contours['INSEE_DEP'].apply(est_mono_departement)

In [ ]:
"""OBTENTION DE gdf_epci_metropole IGNORANT LES EPCI HORS FRANCE METROPOLITAINE"""

def est_metropole(liste_departements):
    """
    Exclut les DOM-TOM et la Corse, ne conserve que la métropole.
    """
    if isinstance(liste_departements, str):
        try:
            liste_departements = ast.literal_eval(liste_departements)
        except Exception:
            return False  # La chaîne n'était pas une liste valide

    # Filtrage : ignorer les départements trop longs
    liste_departements = [str(dep).strip() for dep in liste_departements if len(str(dep).strip()) <= 3]

    for dep in liste_departements:
        # Exclure les codes contenant des lettres (ex: 2A ou 2B pour la Corse)
        if any(c.isalpha() for c in dep):
            return False

        try:
            if int(dep) >= 970:
                return False
        except ValueError:
            return False  # On exclut si ce n'est pas un entier valide

    return True


In [ ]:
gdf_epci_metropole = df_epci_contours[df_epci_contours['INSEE_DEP'].apply(est_metropole)]

In [ ]:
# Vérifier le type des données dans la colonne geometry
print(type(gdf_epci_metropole['geometry'].iloc[0]))

# Si les données sont déjà des objets géométriques, ne pas les convertir
# Sinon, convertir seulement si c'est une chaîne de caractères
def convert_if_string(geom):
    if isinstance(geom, str):
        return wkt.loads(geom)
    return geom

gdf_epci_metropole['geometry'] = gdf_epci_metropole['geometry'].apply(convert_if_string)

# Créer un GeoDataFrame
gdf_epci_metropole = gpd.GeoDataFrame(gdf_epci_metropole, geometry='geometry')

# Tracer les contours des EPCI
fig, ax = plt.subplots(figsize=(10, 10))
gdf_epci_metropole.plot(ax=ax, edgecolor='black', facecolor='none')
plt.title('Contours des EPCI')
plt.axis('off')
plt.show()

In [ ]:
gdf_epci_metropole.sample()

In [ ]:
# Fonction pour convertir les chaînes de caractères en listes
def convert_to_list(value):
    if isinstance(value, str):
        # Supprime les caractères indésirables et divise la chaîne
        cleaned = value.replace('[', '').replace(']', '').replace("'", '').replace('"', '')
        # Divise la chaîne en éléments individuels
        items = [item.strip() for item in cleaned.split(',') if item.strip()]
        return items
    elif isinstance(value, list):
        return value
    return []

# 1. Convertir les colonnes de codes INSEE en listes
print("Conversion des codes INSEE en listes...")
gdf_epci_metropole['INSEE_COM_list'] = gdf_epci_metropole['INSEE_COM'].apply(convert_to_list)
gdf_epci_metropole['INSEE_DEP_list'] = gdf_epci_metropole['INSEE_DEP'].apply(convert_to_list)

# 2. Définir les tranches de distances
distance_bins = [0, 10, 20, 30, 40, 50, float('inf')]
distance_labels = ['<10km', '<20km', '<30km', '<40km', '<50km', '>50km']

# 3. Définir les catégories de transport
transport_categories = {
    'vert': ['1', '2', '3'],
    'jaune': ['6'],
    'rouge': ['4', '5']
}

# Créer un dictionnaire pour stocker les résultats
results = {}

# Boucle sur chaque EPCI
print("Calcul des statistiques par EPCI...")
for idx, epci_row in gdf_epci_metropole.iterrows():
    epci_communes = set(epci_row['INSEE_COM_list'])
    epci_code = epci_row['SIREN_EPCI']
    
    # Initialiser le dictionnaire pour cet EPCI
    results[epci_code] = {}
    
    # Pour chaque catégorie de transport
    for transport_name, transport_codes in transport_categories.items():
        
        # Filtrer les données de mobilité pour cette catégorie de transport
        transport_data = df_mobpro_brut[df_mobpro_brut['TRANS'].isin(transport_codes)]
        
        # Identifier les flux entrants, sortants et internes
        entrants = transport_data[
            (~transport_data['DCLT'].isin(epci_communes)) & 
            (transport_data['COMMUNE'].isin(epci_communes))
        ]
        
        sortants = transport_data[
            (transport_data['DCLT'].isin(epci_communes)) & 
            (~transport_data['COMMUNE'].isin(epci_communes))
        ]
        
        internes = transport_data[
            (transport_data['DCLT'].isin(epci_communes)) & 
            (transport_data['COMMUNE'].isin(epci_communes))
        ]
        
        # Calculer les statistiques par tranche de distance
        for i, (lower, upper) in enumerate(zip(distance_bins[:-1], distance_bins[1:])):
            label = distance_labels[i]
            
            # Flux entrants
            entrants_bin = entrants[(entrants['distance_km'] > lower) & (entrants['distance_km'] <= upper)]
            results[epci_code][f'nb_entrant_{label}_{transport_name}'] = entrants_bin['IPONDI'].sum()
            results[epci_code][f'distance_entrant_{label}_{transport_name}'] = (entrants_bin['distance_km'] * entrants_bin['IPONDI']).sum() / entrants_bin['IPONDI'].sum() if entrants_bin['IPONDI'].sum() > 0 else 0
            
            # Flux sortants
            sortants_bin = sortants[(sortants['distance_km'] > lower) & (sortants['distance_km'] <= upper)]
            results[epci_code][f'nb_sortant_{label}_{transport_name}'] = sortants_bin['IPONDI'].sum()
            results[epci_code][f'distance_sortant_{label}_{transport_name}'] = (sortants_bin['distance_km'] * sortants_bin['IPONDI']).sum() / sortants_bin['IPONDI'].sum() if sortants_bin['IPONDI'].sum() > 0 else 0
        
        # Flux internes (pas de segmentation par distance)
        results[epci_code][f'nb_intra_{transport_name}'] = internes['IPONDI'].sum()
        results[epci_code][f'distance_intra_{transport_name}'] = (internes['distance_km'] * internes['IPONDI']).sum() / internes['IPONDI'].sum() if internes['IPONDI'].sum() > 0 else 0

# Convertir les résultats en DataFrame
print("Conversion des résultats en DataFrame...")
results_df = pd.DataFrame.from_dict(results, orient='index')

# Joindre les résultats au GeoDataFrame original
print("Jointure des résultats au GeoDataFrame...")
gdf_epci_metropole_enrichi = gdf_epci_metropole.merge(results_df, left_on='SIREN_EPCI', right_index=True, how='left')

# Afficher un aperçu des colonnes ajoutées
print("\nColonnes ajoutées:")
new_columns = [col for col in gdf_epci_metropole_enrichi.columns if col not in gdf_epci_metropole.columns]
print(f"Nombre de nouvelles colonnes: {len(new_columns)}")
print(new_columns[:10])  # Afficher les 10 premières nouvelles colonnes

print("\nAperçu du GeoDataFrame enrichi:")
print(gdf_epci_metropole_enrichi.head())

# Le GeoDataFrame enrichi est maintenant prêt à être utilisé
# Il contient toutes les statistiques demandées par EPCI